<a href="https://colab.research.google.com/github/Nithyarajoman/Machine-learning-Trial/blob/main/Functions%20for%20MAC%20calculation%20in%20a%20hierarchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras-flops # Install this for setting an inbuilt function

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
# from keras_flops import get_flops

In [4]:
# Read data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float')/255

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
# Create Model
model = models.Sequential(
    [        
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

In [6]:
# Train
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=15, batch_size=128, validation_split=0.1 )
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/15
422/422 [==============================] - 24s 56ms/step - loss: 0.5462 - accuracy: 0.8079 - val_loss: 0.6069 - val_accuracy: 0.8150
Epoch 2/15
422/422 [==============================] - 23s 55ms/step - loss: 0.3697 - accuracy: 0.8677 - val_loss: 0.3306 - val_accuracy: 0.8800
Epoch 3/15
422/422 [==============================] - 23s 55ms/step - loss: 0.3273 - accuracy: 0.8821 - val_loss: 0.2937 - val_accuracy: 0.8907
Epoch 4/15
422/422 [==============================] - 23s 56ms/step - loss: 0.3032 - accuracy: 0.8901 - val_loss: 0.2997 - val_accuracy: 0.8888
Epoch 5/15
422/422 [==============================] - 23s 55ms/step - loss: 0.2893 - accuracy: 0.8942 - val_loss: 0.2787 - val_accuracy: 0.8997
Epoch 6/15
422/422 [==============================] - 23s 55ms/step - loss: 0.2795 - accuracy: 0.8991 - val_loss: 0.2657 - val_accuracy: 0.9035
Epoch 7/15
422/422 [==============================] - 24s 56ms/step - loss: 0.2648 - accuracy: 0.9038 - val_loss: 0.2752 - val_accuracy:

In [7]:
# Creating a different model using some output layers from the trained network (for hierarchy)

all_layers = [layer.output for layer in [model.get_layer(index=0),
                                        model.get_layer(index=1),
                                        model.get_layer(index=2),
                                        model.get_layer(index=3),
                                        model.get_layer(index=4),
                                        model.get_layer(index=5)]]

all_model = tf.keras.Model(inputs=model.inputs, outputs=all_layers)
all_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        288       
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18432     
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 64)         256       
 hNormalization)                                             

In [28]:
# Creating a model with hieracrchy

model_hierarchy = models.Sequential(
    [        
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        
        models.Sequential(
            [
                layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(14, 14, 32),
                      strides=(2,2), padding="same",
                      use_bias=False),
                layers.BatchNormalization(),
                layers.Activation(activation="relu"),
            ]
        ),

        layers.Conv2D(64, kernel_size=(3, 3),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

In [29]:
model_hierarchy.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 14, 14, 32)        288       
                                                                 
 batch_normalization_25 (Bat  (None, 14, 14, 32)       128       
 chNormalization)                                                
                                                                 
 activation_25 (Activation)  (None, 14, 14, 32)        0         
                                                                 
 sequential_7 (Sequential)   (None, 7, 7, 32)          9344      
                                                                 
 conv2d_27 (Conv2D)          (None, 4, 4, 64)          18432     
                                                                 
 batch_normalization_27 (Bat  (None, 4, 4, 64)         256       
 chNormalization)                                     

In [24]:
# Calculating MACs of complete model using a buil in function

flops = get_flops(model, batch_size=1)
flops = get_flops(model, batch_size=1)
print(f"FLOPS for the entire model: {flops}")

FLOPS for the entire model: 2001404


In [32]:
for li in model.layers:
  cfg = li.get_config()
  print(li.name)
  print(cfg)

conv2d
{'name': 'conv2d', 'trainable': True, 'batch_input_shape': (None, 28, 28, 1), 'dtype': 'float32', 'filters': 32, 'kernel_size': (3, 3), 'strides': (2, 2), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': False, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
batch_normalization
{'name': 'batch_normalization', 'trainable': True, 'dtype': 'float32', 'axis': ListWrapper([3]), 'momentum': 0.99, 'epsilon': 0.001, 'center': True, 'scale': True, 'beta_initializer': {'class_name': 'Zeros', 'config': {}}, 'gamma_initializer': {'class_name': 'Ones', 'config': {}}, 'moving_mean_initializer': {'class_name': 'Zeros', 'config': {}}, 'moving_variance_initializer': {'class_name': 'Ones', 'config'

In [ ]:
for l in model.layers:
  if ('InputLayer' in str(l)):
            i_shape = l.input.get_shape()[1:4].as_list()
            o_shape = i_shape
  print("Input:%f", {i_shape})



In [54]:
# Calculating FLOPs for each layer of the model
def flops_per_layer(model):
    
    # To print the output in the tabular form:
    print('%25s | %16s | %16s | %16s | %16s | %6s | %6s' % (
        'Layer', 'Input Shape', 'Output Shape', 'Kernel Size', 'Filters', 'Strides', 'MACs'))
    print('*' * 170)
    
    # Initialising:
    t_macc = 0

    # Iterarting through every layer/ model in the main model
    for l in model.layers:

        o_shape, i_shape, strides, ks, filters = ['', '', ''], ['', '', ''], [1, 1], [0, 0], [0, 0]
        
        macc = 0
        name = l.name

        if ('Sequential' in str(l)):
            macc =  flops_per_layer(l)

        if ('InputLayer' in str(l)):
            i_shape = l.input.get_shape()[1:4].as_list()
            o_shape = i_shape

        if ('Reshape' in str(l)):
            i_shape = l.input.get_shape()[1:4].as_list()
            o_shape = l.output.get_shape()[1:4].as_list()
       

        if ('Dense' in str(l)):
            print(l.input)
            i_shape = l.input.shape[1:4].as_list()[0]
            o_shape = l.output.shape[1:4].as_list()
            macc = (o_shape[0] * i_shape)

        
        if ('Conv2D ' in str(l) and 'DepthwiseConv2D' not in str(l) and 'SeparableConv2D' not in str(l)):
            strides = l.strides
            ks = l.kernel_size
            filters = l.filters
            i_shape = l.input.get_shape()[1:4].as_list()
            o_shape = l.output.get_shape()[1:4].as_list()

            if (filters == None):
                filters = i_shape[2]

            macc = ((filters * ks[0] * ks[1] * i_shape[2]) * (
                    (i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

        if ('Conv2D ' in str(l) and 'DepthwiseConv2D' in str(l) and 'SeparableConv2D' not in str(l)):
            strides = l.strides
            ks = l.kernel_size
            filters = l.filters
            i_shape = l.input.get_shape()[1:4].as_list()
            o_shape = l.output.get_shape()[1:4].as_list()

            if (filters == None):
                filters = i_shape[2]

            macc = ((ks[0] * ks[1] * i_shape[2]) * ((i_shape[0] / strides[0]) * (i_shape[1] / strides[1]))) 
             
        
        # t_macc += macc        
        print('%25s | %16s | %16s | %16s | %16s | %6s | %5.4f' % (
            name, str(i_shape), str(o_shape), str(ks), str(filters), str(strides), macc))
    
    
    # print('\nTotal MACs for the entire model: %10.8f\n' % (t_macc))

    return

In [ ]:
flops_per_layer(model_hierarchy)

In [66]:
# Function to calculate return MAC from each layer

def mac_each_layer(l):
    o_shape, i_shape, strides, ks, filters = ['', '', ''], ['', '', ''], [1, 1], [0, 0], [0, 0]        
    macc = 0
    
    if ('Dense' in str(l)):
        print(l.input)
        i_shape = l.input.shape[1:4].as_list()[0]
        o_shape = l.output.shape[1:4].as_list()
        macc = (o_shape[0] * i_shape)

    
    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' not in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((filters * ks[0] * ks[1] * i_shape[2]) * (
                (i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((ks[0] * ks[1] * i_shape[2]) * ((i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    return macc



In [101]:
# Function to pass layers to the calculating function and to get the macc of each layer

def summary_MAC(model,table=True):
  if (table == True):
        print('%25s | %6s' % (
            'Layer Name', 'MACs'))
        print('*' * 100)
  t_macc = 0

  for l in model.layers:
    if ('Sequential' in str(l)):
      for ib_layer in l.layers:
        mac_obtained = mac_each_layer(ib_layer)
        print('%25s(%5s) | %5.4f' % (
                l.name,ib_layer.name, mac_obtained ))
        t_macc += mac_obtained  

    else:
      mac_obtained = mac_each_layer(l)
      print('%25s | %5.4f' % (
                l.name, mac_obtained ))
      
    t_macc += mac_obtained
  print('\nTotal MACs operations in the complete model: %10.8f\n' % (t_macc))
  return

In [102]:
summary_MAC(model_hierarchy)

               Layer Name |   MACs
****************************************************************************************************
                conv2d_28 | 56448.0000
   batch_normalization_28 | 0.0000
            activation_28 | 0.0000
             sequential_9(conv2d_29) | 451584.0000
             sequential_9(batch_normalization_29) | 0.0000
             sequential_9(activation_29) | 0.0000
                conv2d_30 | 225792.0000
   batch_normalization_30 | 0.0000
            activation_30 | 0.0000
                flatten_6 | 0.0000
                dropout_6 | 0.0000
KerasTensor(type_spec=TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), name='dropout_6/Identity:0', description="created by layer 'dropout_6'")
                  dense_6 | 10240.0000

Total MACs operations in the complete model: 744064.00000000

